# Construct a MLP to identify primary vertices 

In this exercise we will calulate the primary vertex from simulated tracks in a detector produced from colliding bunches of protons. We assume that there is one hard proton-proton interaction in an event, at the point we call the primary vertex. These tracks are reconstructed from the hard proton-proton interaction and tend to have higher p<sub>T</sub> values. The majority of the tracks in the event are not from the hard proton-proton interaction, but are from soft proton-proton interactions (pileup) and typically have lower p<sub>T</sub>

Lets simulate the data. There is no need to understand the details of the data simulation. All you need to know is that the interaction region spans from 0 to 1 in z where for each event there are tracks from pileup events uniformly distributed. For each event there is also a primary vertex at a random position in z which generates tracks around around this point with a higher p<sub>T</sub> distribution

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras

def generateCollisonTracks(numEvents=10000):

    maxTracks = 100
    aveNumPrimaryVertexTracks = 10
    stdNumPrimaryVertexTracks = 3
    vertexResolution = 0.1
    meanPrimaryTrackPt = 50
    stdPrimaryTrackPt = 10
    meanPileUpTrackPt = 10
    stdPileUpTrackPt = 10

    #Allocate memory for dataset
    ds = np.zeros((numEvents,maxTracks,2))
    primary_vertices = np.zeros((numEvents))

    #Be suspicious of loops with libraries that employ vectorisation
    # Not worried about poor performance here
    for ievent in range(numEvents):
        y = np.random.uniform(0.2,0.8)
        numPrimaryVertexTracks = np.random.normal(aveNumPrimaryVertexTracks, stdNumPrimaryVertexTracks, 1).astype(int)[0]
        
        #We want at least 1 track
        if numPrimaryVertexTracks < 1:
            numPrimaryVertexTracks = 1
            
        primaryTracks_z = np.random.normal(y, vertexResolution, numPrimaryVertexTracks)
        primaryTracks_pt = np.random.normal(meanPrimaryTrackPt, stdPrimaryTrackPt, numPrimaryVertexTracks)

        #Numberof tracks from soft collision
        numPileupTracks = maxTracks - numPrimaryVertexTracks

        pileUpTracks_z = np.random.uniform(0.0,1.0, numPileupTracks)
        pileUpTracks_pt = np.random.normal(meanPileUpTrackPt, stdPileUpTrackPt, numPileupTracks)

        #Assign generated events to dataset
        ds[ievent,:numPrimaryVertexTracks,0] =  primaryTracks_z[:]
        ds[ievent,:numPrimaryVertexTracks,1] =  primaryTracks_pt[:]
        ds[ievent,numPrimaryVertexTracks:,0] =  pileUpTracks_z[:]
        ds[ievent,numPrimaryVertexTracks:,1] =  pileUpTracks_pt[:]

        primary_vertices[ievent] = y

        #Shuffle the tracks
        shuffledIndex = np.arange(maxTracks)
        np.random.shuffle(shuffledIndex)
        ds[ievent] = ds[ievent,shuffledIndex]

    return primary_vertices, ds


Generate data by calling the generateCollisonTracks function. This function returns an array with the postion of the primary vertex for each event (p_vtx) and an array with the track information for each event (z and p<sub>T</sub>)

In [ ]:
p_vtx, trackData = generateCollisonTracks(10000)

The function generateCollisonTracks generates 10000 events (or whatever number your request). Each event will consist of 100 arrays, each representing a track. Each track array holds 2 variables, the z value of the track extrapolated to the beam line and the p<sub>T</sub> associated with the track

Take a look at the layout of the data using the .shape method

In [ ]:
trackData.shape

Look at the data itselt. Say the first event of the track data

In [ ]:
trackData[0]

We also have the actual values of the primary vertex which we can train against. Look at the first 10 events

In [ ]:
p_vtx[0:10]

Look at the disribution of tracks in z. Not very informative just plotting the tracks in z. However you should see that the distribution is flat. There is a cluster of tracks around the primary vertex in there somewhere

In [ ]:
plt.hist(trackData[0,:,0], bins=100)

### Exercise

Use a simple MLP and train on the z and p<sub>T</sub> of the tracks using the trackData array to calculate the primary vertex found in the p_vtx array
    
 First separate the data into a training, validation and test dataset

In [ ]:
endTrainingIndex = 8000
endValidationIndex = 9000

#Split into training and validation samples
data_train, data_val, data_test =  trackData[:endTrainingIndex], trackData[endTrainingIndex:endValidationIndex],trackData[endTrainingIndex:endValidationIndex]
y_train, y_val , y_test =  p_vtx[:endTrainingIndex], p_vtx[endTrainingIndex:endValidationIndex], p_vtx[endTrainingIndex:endValidationIndex]


Construct a simple MLP

In [ ]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(100, 2)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(1)
])

model.summary()

In [ ]:
# Set loss function and optimiser
loss_fn = tf.keras.losses.MeanSquaredError()

model.compile(loss=loss_fn, optimizer="adam")

In [ ]:
num_epochs = 400
#Not worried about memory or local minima
batchSize = len(data_train)

#Train on data
history = model.fit(data_train, y_train,
          validation_data=(data_val, y_val),
          batch_size=len(y_train),
          epochs=num_epochs)